In [1]:
import numpy as np
import torch
import cv2
import os
import voc12.data
import scipy.misc
import importlib
from torch.utils.data import DataLoader
import torchvision
from tool import imutils, pyutils, visualization
import argparse
from PIL import Image
import torch.nn.functional as F
import pandas as pd

In [2]:
model = getattr(importlib.import_module("network.resnet38_SEAM"), 'Net')()
model.load_state_dict(torch.load("resnet38_SEAM.pth"))
model.eval()
model.cuda()

Net(
  (conv1a): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (b2): ResBlock(
    (bn_branch2a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv_branch1): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  )
  (b2_1): ResBlock(
    (bn_branch2a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), pad

In [3]:
infer_dataset = voc12.data.VOC12ClsDatasetMSF("voc12/bb_list.txt", voc12_root="/home/bnair/data/chest14",
                                                  scales=[0.5, 1.0, 1.5, 2.0],
                                                  inter_transform=torchvision.transforms.Compose(
                                                       [np.asarray,
                                                        model.normalize,
                                                        imutils.HWC_to_CHW]))

infer_data_loader = DataLoader(infer_dataset, shuffle=False, num_workers=8, pin_memory=True)

In [6]:
n_gpus = torch.cuda.device_count()
model_replicas = torch.nn.parallel.replicate(model, list(range(n_gpus)))

In [4]:
import PIL
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [ ]:
from torchvision.transforms.functional import to_pil_image
from torchcam.utils import overlay_mask
import torchvision.transforms as T

# get list of images in outcam/ folder
outcam_img_list = os.listdir("outcam/")
# print(outcam_img_list)

for iter, (img_name, img_list, label) in enumerate(infer_data_loader):
    img_name = img_name[0]; label = label[0]

    if img_name + '.png' not in outcam_img_list:
        print("{} not in outcam/".format(img_name))
        img_path = voc12.data.get_img_path(img_name, "/home/bnair/data/chest14")
        orig_img = np.asarray(Image.open(img_path))
        orig_img_size = orig_img.shape[:2]
        def _work(i, img):
                with torch.no_grad():
                    with torch.cuda.device(i%n_gpus):
                        pred= model_replicas[i%n_gpus](img.cuda())
                        img = tensor_to_image(pred[0][0][0].cpu())
                        im1 = img.save(os.path.join('/home/bnair/SEAM/outcam', img_name + '.png'))
                        return im1

        thread_pool = pyutils.BatchThreader(_work, list(enumerate(img_list)),
                                                batch_size=2, prefetch_size=0, processes=1)
        
        cam_list = thread_pool.pop_results()

    # else:
        # print("Already processed: " + img_name)

In [5]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [6]:
from chitra.image import Chitra
from PIL import Image
from tqdm import tqdm

bboxes = pd.read_csv('./voc12/BBox_List_2017.csv')
localization_images = {}
for index, row in tqdm(bboxes.iterrows()):
    img_name = row['Image Index']
    if img_name[1:] in os.listdir('/home/bnair/SEAM/outcam'):
        box = [[row['x'], row['y'], row['h'], row['w']]]
        label = [row['Finding Label']]
        image = Chitra('/home/bnair/data/chest14/'+img_name, labels=label, bboxes=box, box_format='xyhw')
        # image.resize_image_with_bbox((224, 224))
        # localization_images[img_name] = {
        #     'label': row['Finding Label'],
        #     'xA': (image.bboxes[0].x1/1024)*512,
        #     'yA': (image.bboxes[0].y1/1024)*512,
        #     'xB': (image.bboxes[0].x2/1024)*512,
        #     'yB': (image.bboxes[0].y2/1024)*512,
        # }
        localization_images[img_name] = {
            'label': row['Finding Label'],
            'xA': (image.bboxes[0].x1),
            'yA': (image.bboxes[0].y1),
            'xB': (image.bboxes[0].x2),
            'yB': (image.bboxes[0].y2),
        }

984it [00:00, 1144.83it/s]


In [7]:
def get_class_from_image(image_file):
    bboxes = pd.read_csv('./BBox_List_2017.csv')
    img_row = bboxes[bboxes['Image Index'] == image_file]
    return img_row['Finding Label'].values[0]

In [8]:
scores = {}
diseases_available = bboxes['Finding Label'].unique()
for disease in diseases_available:
    scores[disease] = {
        'count': bboxes[bboxes['Finding Label'] == disease].shape[0],
        'evaluated': 0,
        'total_iou': 0,
        'total_cdice': 0,
    }

In [9]:
for img_name in os.listdir('/home/bnair/SEAM/outcam'):
    # print(img_name)
    try:
        img = cv2.imread(os.path.join('/home/bnair/SEAM/outcam', img_name))
        img = cv2.resize(img, (1024, 1024))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        cv2.imwrite("thresh.png", thresh)
        contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        bboxes = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if x == 0 or x == 1024 - 1 or y == 0 or y == 1024 - 1:
                continue
            bboxes.append((x,y,w,h))
            # bboxes.append([x, y, x+w, y+h])
        if len(bboxes) == 0:
            scores[localization_images["0"+img_name]['label']]['evaluated'] += 1
            continue
        ious = []
        iou_bb = {}
        gt_XA, gt_YA, gt_XB, gt_YB = 0, 0, 0, 0
        xA, yA, xB, yB = 0, 0, 0, 0
        for bbox in bboxes:
            xA, yA, xB, yB = bbox
            label = localization_images["0"+img_name]['label']
            gt_XA = localization_images["0"+img_name]['xA']
            gt_YA = localization_images["0"+img_name]['yA']
            gt_XB = localization_images["0"+img_name]['xB']
            gt_YB = localization_images["0"+img_name]['yB']
            # draw both the boundiug boxes and the label on the image
            ious.append(bb_intersection_over_union([int(xA), int(yA), int(xB), int(yB)], [int(gt_XA), int(gt_YA), int(gt_XB), int(gt_YB)]))
            iou_bb[bb_intersection_over_union([int(xA), int(yA), int(xB), int(yB)], [int(gt_XA), int(gt_YA), int(gt_XB), int(gt_YB)])] = bbox
        # print(max(ious))
        scores[label]['total_iou'] += max(ious)
        # scores[label]['bbox'] = iou_bb[max(ious)] 
        scores[label]['evaluated'] += 1
        xA, yA, xB, yB = iou_bb[max(ious)]
        img = cv2.imread(os.path.join('/home/bnair/data/chest14', "0" + img_name))
        # img = cv2.resize(img, (512, 512))
        cv2.rectangle(img, (int(xA), int(yA)), (int(xB), int(yB)), (0, 0, 255), 2)
        cv2.rectangle(img, (int(gt_XA), int(gt_YA)), (int(gt_XB), int(gt_YB)), (0, 255, 0), 2)
        cv2.imwrite(os.path.join('/home/bnair/SEAM/outcamgtbb', label + '_' + img_name), img)
    except:
        print(img_name)

0008554_009.png
0020774_000.png


In [10]:
scores

{'Atelectasis': {'count': 180,
  'evaluated': 157,
  'total_iou': 0.0,
  'total_cdice': 0},
 'Cardiomegaly': {'count': 146,
  'evaluated': 135,
  'total_iou': 0.0,
  'total_cdice': 0},
 'Effusion': {'count': 153,
  'evaluated': 114,
  'total_iou': 0.0,
  'total_cdice': 0},
 'Infiltrate': {'count': 123,
  'evaluated': 101,
  'total_iou': 0.0,
  'total_cdice': 0},
 'Mass': {'count': 85, 'evaluated': 78, 'total_iou': 0.0, 'total_cdice': 0},
 'Nodule': {'count': 79, 'evaluated': 74, 'total_iou': 0.0, 'total_cdice': 0},
 'Pneumonia': {'count': 120,
  'evaluated': 118,
  'total_iou': 0.0,
  'total_cdice': 0},
 'Pneumothorax': {'count': 98,
  'evaluated': 98,
  'total_iou': 0.0,
  'total_cdice': 0}}